In [54]:
import starfile
import glob
# import argparse
import plotly.express as px

In [65]:
files = glob.glob('../samples/input/*_particles.star')
# df = starfile.read(file)
# particles = df['particles']
dfs = []
for f in sorted(files):
    df = starfile.read(f)
    dfs.append(df['particles'])

In [66]:
print(sorted(files))
# print(dfs[0])

['../samples/input/01_particles.star', '../samples/input/02_particles.star', '../samples/input/03_particles.star', '../samples/input/04_particles.star']


In [67]:
plot_x = 'rlnCoordinateX'
plot_y = 'rlnCoordinateY'
plot_z = 'rlnClassNumber'
frame_name = 'Iteration'

In [93]:
fig = go.Figure(frames=[go.Frame(data=go.Scatter(
                                            x=df[plot_x], 
                                            y=df[plot_y], 
                                            mode='markers',
                                            marker=dict(
                                                color=df[plot_z],
                                                colorscale='Viridis',
                                                size=5,
                                                showscale=True
                                            ),
                                            text = ['{:0.3f}'.format(i) for i in df[plot_z]],
                                            hovertemplate = plot_z + ': %{text}',
                                            ),
                                            name=str(k)
                                            )
                    for k, df in enumerate(dfs)])

# Add data to be displayed before animation starts
fig.add_trace(go.Scatter(
                x=dfs[0][plot_x], 
                y=dfs[0][plot_y], 
                mode='markers',
                marker=dict(
                    color=dfs[0][plot_z],
                    colorscale='Viridis',
                    size=5,
                    showscale=True
                ),
                text = ['{:0.3f}'.format(i) for i in dfs[0][plot_z]],
                hovertemplate = plot_z + ': %{text}',
                name='',
            ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         width=600,
         height=600,
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(500)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)


fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
  )

fig.update_layout(
    xaxis_title=plot_x,
    yaxis_title=plot_y,
#     width=700,
    height=700,
    autosize=False,
    margin=dict(t=50, b=0, l=0, r=0),
    annotations=[
        dict(text="Color: " + plot_z, x=0, xref="paper", y=1.05, yref="paper",
                         align="left", showarrow=False)
    ]    
)

fig.show()